In [ ]:
# ================================
# Underwater Object Detection — Custom YOLO Model
# Video / Webcam Inference (No cv2.imshow)
# ================================

import os
import cv2
import time
from ultralytics import YOLO
from IPython.display import display, clear_output
from PIL import Image

# === CONFIGURATION ===
#MODEL_PATH = "//home/bodhdipta/Downloads/Minor-Project//Final-Project/best-50.pt"
MODEL_PATH = "yolov8s.pt"
VIDEO_PATH =  None # or None for webcam
CONF_THRESHOLD = 0.45
# ======================


def load_model(path):
    """Load your pre-trained YOLO model."""
    if os.path.exists(path):
        print(f"[INFO] Loading custom trained model from: {path}")
        return YOLO(path)
    else:
        print(f"[ERROR] Model not found at: {path}")
        exit(1)


def show_frame(frame):
    """Display frame in Jupyter notebook instead of cv2.imshow."""
    clear_output(wait=True)
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    display(Image.fromarray(img_rgb))


def run_video_detection(model, conf=0.45, video_path=None):
    cap = cv2.VideoCapture(0 if video_path is None else video_path)
    source_name = "webcam" if video_path is None else video_path
    print(f"[INFO] Starting detection from {source_name}...")

    if not cap.isOpened():
        print("[ERROR] Could not open video source.")
        return

    prev_time = time.time()
    frame_count = 0

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("[INFO] End of video or failed to read frame.")
                break

            # Run detection
            results = model.predict(frame, conf=conf, verbose=False)
            annotated = results[0].plot()

            # FPS calculation
            frame_count += 1
            if frame_count % 5 == 0:
                now = time.time()
                fps = 5 / (now - prev_time)
                prev_time = now
                cv2.putText(annotated, f"FPS: {fps:.1f}", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            # Display frame (no GUI required)
            show_frame(annotated)

            # Stop on keyboard interrupt inside notebook (optional)
            # -> Just press the stop button in notebook

    finally:
        cap.release()
        print("[INFO] Detection ended.")


if __name__ == "__main__":
    model = load_model(MODEL_PATH)
    run_video_detection(model, conf=CONF_THRESHOLD, video_path=VIDEO_PATH)


[INFO] Detection ended.


KeyboardInterrupt: 